In [1]:
import keras.backend as K
import tensorflow as tf
from keras import Input
from keras.engine import Model
from keras.models import model_from_json, Sequential
from tensorflow.python.framework import sparse_tensor
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.layers import TimeDistributed, Activation, Dense, Dropout, GRU, Input, Bidirectional, LSTM, Masking, GaussianNoise
from keras.optimizers import Adam
from keras.preprocessing import sequence
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
import time
import warnings
import os

In [2]:
print(tf.__version__)

2.3.1


In [18]:
def equal(tensor1, tensor2):
    for i, j in zip(tensor1.reshape(-1), tensor2.reshape(-1)):
        if abs(i - j) > 0.001:
            return False
    return True

In [19]:
new_model = Sequential()


new_model.add(Input(name='input', batch_size=1, shape=(925, 3)) )
new_model.add(Dense(8))
new_model.add(LSTM(64, return_sequences=True))
new_model.add(LSTM(64, return_sequences=True))
new_model.add(LSTM(64, return_sequences=True))
new_model.add(LSTM(64, return_sequences=True))
new_model.add(Activation('softmax', name='softmax'))


new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (1, 925, 8)               32        
_________________________________________________________________
lstm_12 (LSTM)               (1, 925, 64)              18688     
_________________________________________________________________
lstm_13 (LSTM)               (1, 925, 64)              33024     
_________________________________________________________________
lstm_14 (LSTM)               (1, 925, 64)              33024     
_________________________________________________________________
lstm_15 (LSTM)               (1, 925, 64)              33024     
_________________________________________________________________
softmax (Activation)         (1, 925, 64)              0         
Total params: 117,792
Trainable params: 117,792
Non-trainable params: 0
________________________________________________

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
#converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.allow_custom_ops=True
tflite_model = converter.convert()



open("tf_test.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp4ghk5_eb/assets


INFO:tensorflow:Assets written to: /tmp/tmp4ghk5_eb/assets


476052

In [30]:
interpreter = tf.lite.Interpreter(model_path="tf_test.tflite")
output_details = interpreter.get_output_details()
inputs = interpreter.get_input_details()
interpreter.allocate_tensors()

In [31]:
data = np.array(np.random.random_sample(inputs[0]['shape']), dtype=np.float32)
interpreter.set_tensor(inputs[0]['index'], data)


#### Testing converted model model 
The output differs across different runs

In [32]:
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
interpreter.invoke()
output2 = interpreter.get_tensor(output_details[0]['index'])
equal(output,output2)

False

#### Sometimes it works after few runs.

In [33]:
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
interpreter.invoke()
output2 = interpreter.get_tensor(output_details[0]['index'])
equal(output,output2)

True

#### Non-converted TensorFlow always yields the same output

In [29]:
output3 = new_model.predict(data)
output4 = new_model.predict(data)
equal(output3,output4)

True